In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math
import scipy
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

import pathlib
import sklearn.metrics
from scipy.stats import gaussian_kde
import matplotlib.cm as cm
from dataset_fetcher import download_uci_adult

import random

In [2]:
def model_outcomes_coal(model, coal, local_obs, ref_point, seed=0):
    
    ## this function outputs a list of all model predictions for a single coalition
    # the input is the coalition of interest as a binary vector eg. [1,1,0,1]
    # ref point is the entire reference distirbution (here we take x_train)
    # local_obs is the instance we aim to explain
    

    # we consider all 2**n coalitions and all n reference points
    # get constants
    n, d = np.shape(ref_point)
    
    # train imputation algorithm for conditional references   
    # create an "all_imputed_coalitions" 3D matrix with all "artificial inputs" we get by imputing
    coalitions = np.array([[int(i) for i in '0'*((d)-len(bin(j))+2) + bin(j)[2:]] for j in range(2**d)]) 
    all_imputed_coalitions = np.zeros((n, d))
    
    imp = IterativeImputer(max_iter=100, random_state=0, sample_posterior=True)
    imp.fit(ref_point) #imputer learns from marginal distribution
    
    
    
     ## below we build the concatenated inputs using conditional imputation for dropped features and averages
    for k in range(n):#reference point index     
        vect=(1-coal) 
        vect = vect.astype('float')
        vect[vect == 1] = 'nan'    # vect is a binary vector of being "absent" ('nan') or "present" (1)
        # impute conditionally with Bayesian Ridge MICE 
        imputed_coalitions = coal * local_obs + vect #either local obs value or 'nan'
        imputed_coalitions = imp.transform(imputed_coalitions.reshape(1,-1))       
        all_imputed_coalitions[k, :] = imputed_coalitions.reshape(d)
    
    #we ultimately return the model predictions
    return model.predict_proba(all_imputed_coalitions)[:,1]


In [3]:
data_dir = pathlib.Path("uci") ## add your path here
train_df, test_df = download_uci_adult(data_dir)

data=pd.concat([train_df, test_df], axis=0)
n=np.shape(data)[0]

random.seed(0)
subsample_initial=random.sample(range(0, n-1), 5000)

train_df=data.iloc[subsample_initial]

train_df=train_df[['fnlwgt','age','race','occupation','marital-status','relationship','capitalgain','native-country','class']]
sample_weights = train_df['fnlwgt']
y_train = train_df['class']
x_train = train_df.drop(columns=['class', 'fnlwgt'])


/home/clivio/ox3/projects/p220923_pwshap_to_aistats/e230529_github_code/notebooks/dataset_fetcher.py:55: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  test_df['class'] = test_df['class'].str.replace('.', '')
/home/clivio/ox3/projects/p220923_pwshap_to_aistats/e230529_github_code/notebooks/dataset_fetcher.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_df = train_df.append(test_df)


In [4]:
###making race binary: white=1
x_train['race'].value_counts()
x_train[['race']]=x_train[['race']].replace(['Black', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other'], 0)
x_train[['race']]=x_train[['race']].replace(['White'], 1)
x_train['race'].value_counts()

1    4264
0     736
Name: race, dtype: int64

In [5]:
##making occupation binary : tertiary sector = 1
x_train['occupation'].value_counts()
x_train[['occupation']]=x_train[['occupation']].replace(['Prof-specialty', 'Exec-managerial',
                                                         'Other-service', 'Tech-support', '?', 'Sales','Machine-op-inspct'], 1)
x_train[['occupation']]=x_train[['occupation']].replace(['Craft-repair', 'Adm-clerical',
                                                         'Transport-moving', 'Handlers-cleaners', 'Farming-fishing',
                                                         'Protective-serv', 'Priv-house-serv', 'Armed-Forces'], 0)

In [6]:
# pre processing
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder(dtype=int)
x_train[['native-country','marital-status','relationship']] = ord_enc.fit_transform(x_train[['native-country','marital-status','relationship']])


In [7]:
# fit and score the model
outcome = RandomForestClassifier(n_estimators=500, class_weight='balanced', random_state=0)
outcome.fit(x_train, y_train, sample_weight=sample_weights)
print(sklearn.metrics.accuracy_score(y_train, outcome.predict(x_train)))
print(sklearn.metrics.roc_auc_score(y_train, outcome.predict_proba(x_train)[:, 1]))


0.8346
0.9281609157635291


## Treatment = Race (mediation analysis) 4

In [8]:
propensity_race = RandomForestClassifier(n_estimators=500, class_weight="balanced", random_state=0)
propensity_race.fit(x_train.drop(columns=['race']), x_train['race'])
print(sklearn.metrics.accuracy_score(x_train['race'], propensity_race.predict(x_train.drop(columns=['race']))))
print(sklearn.metrics.roc_auc_score(x_train['race'], propensity_race.predict_proba(x_train.drop(columns=['race']))[:, 1]))

0.8722
0.9104780798800881


In [9]:
inst=10 # chose your treated instance
print(x_train.iloc[inst,:].values)
print(y_train.iloc[inst])
print(outcome.predict_proba(x_train)[inst,1])

[38  1  1  0  4  0 38]
>50K
0.5364075446395272


In [10]:
# COALITION-SPECIFIC SHAPLEY TERMS: an example (we only want the mean prediction over all reference points ultimately)

model_outcome_all= np.mean(model_outcomes_coal(outcome, np.array([1, 1, 1, 1, 1, 1, 1]), x_train.iloc[inst,:].values, x_train.values, seed=0))
model_outcome_all_but_race= np.mean(model_outcomes_coal(outcome, np.array([1, 0, 1, 1, 1, 1, 1]), x_train.iloc[inst,:].values, x_train.values, seed=0))
model_outcome_all_but_marital= np.mean(model_outcomes_coal(outcome, np.array([1, 1, 1, 0, 1, 1, 1]), x_train.iloc[inst,:].values, x_train.values, seed=0))
model_outcome_all_but_race_but_marital = np.mean(model_outcomes_coal(outcome, np.array([1, 0, 1, 0, 1, 1, 1]), x_train.iloc[inst,:].values, x_train.values, seed=0))

print('model_outcome_all '+str(model_outcome_all))
print('model_outcome_all_but_race '+str(model_outcome_all_but_race))
print('model_outcome_all_but_marital '+str(model_outcome_all_but_marital))
print('model_outcome_all_but_race_but_marital '+str(model_outcome_all_but_race_but_marital))


/home/clivio/miniconda3/envs/pwshap/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/clivio/miniconda3/envs/pwshap/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/clivio/miniconda3/envs/pwshap/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


model_outcome_all 0.5364075446395272
model_outcome_all_but_race 0.45412186137179233
model_outcome_all_but_marital 0.165421556594821
model_outcome_all_but_race_but_marital 0.14769848819642178


/home/clivio/miniconda3/envs/pwshap/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [11]:
# weights : 1 - P(T=1 knowing CS) example:
weight_all_but_race= 1-np.mean(model_outcomes_coal(propensity_race, np.array([1, 1, 1, 1, 1, 1]), x_train.drop(columns=['race']).iloc[inst,:].values, x_train.drop(columns=['race']).values, seed=0))
weight_all_but_race_but_marital=1--np.mean(model_outcomes_coal(propensity_race, np.array([0, 0, 1, 0, 0, 0]), x_train.drop(columns=['race']).iloc[inst,:].values, x_train.drop(columns=['race']).values, seed=0))

print('weight_all_but_race '+str(weight_all_but_race))
print('weight_all_but_race_but_marital '+str(weight_all_but_race_but_marital))


/home/clivio/miniconda3/envs/pwshap/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/clivio/miniconda3/envs/pwshap/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


weight_all_but_race 0.4334032432569791
weight_all_but_race_but_marital 1.6175033666081315


In [12]:
phi_med_by_marital=((model_outcome_all-model_outcome_all_but_race)/weight_all_but_race)-((model_outcome_all_but_marital-model_outcome_all_but_race_but_marital)/weight_all_but_race_but_marital)

print('phi_med_by_marital ' +str(phi_med_by_marital))


phi_med_by_marital 0.17890235624088272


## Treatment = Occupation (bias analysis)

In [13]:
propensity_occ = RandomForestClassifier(n_estimators=500, oob_score=True, class_weight="balanced", random_state=0)
propensity_occ.fit(x_train.drop(columns=['occupation']), x_train['occupation'])
print(sklearn.metrics.accuracy_score(x_train['occupation'], propensity_occ.predict(x_train.drop(columns=['occupation']))))
print(sklearn.metrics.roc_auc_score(x_train['occupation'], propensity_occ.predict_proba(x_train.drop(columns=['occupation']))[:, 1]))

0.7102
0.8113698548655722


In [14]:
model_outcome_all= np.mean(model_outcomes_coal(outcome, np.array([1, 1, 1, 1, 1, 1, 1]), x_train.iloc[inst,:].values, x_train.values, seed=0))
model_outcome_all_but_occup= np.mean(model_outcomes_coal(outcome, np.array([1, 1, 0, 1, 1, 1, 1]), x_train.iloc[inst,:].values, x_train.values, seed=0))
model_outcome_all_but_relationship= np.mean(model_outcomes_coal(outcome, np.array([1, 1, 1, 1, 0, 1, 1]), x_train.iloc[inst,:].values, x_train.values, seed=0))
model_outcome_all_but_relationship_but_occup= np.mean(model_outcomes_coal(outcome, np.array([1, 1, 0, 1, 0, 1, 1]), x_train.iloc[inst,:].values, x_train.values, seed=0))

print('model_outcome_all '+str(model_outcome_all))
print('model_outcome_all_but_occup '+str(model_outcome_all_but_occup))
print('model_outcome_all_but_relationship '+str(model_outcome_all_but_relationship))
print('model_outcome_all_but_relationship_but_occup '+str(model_outcome_all_but_relationship_but_occup))


/home/clivio/miniconda3/envs/pwshap/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/clivio/miniconda3/envs/pwshap/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/clivio/miniconda3/envs/pwshap/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/clivio/miniconda3/envs/pwshap/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


model_outcome_all 0.5364075446395272
model_outcome_all_but_occup 0.3131024532681641
model_outcome_all_but_relationship 0.4435772873515245
model_outcome_all_but_relationship_but_occup 0.34468733150662567


In [15]:
#weights
# weights : 1 - P(T=1 knowing CS)
weight_all_but_occ = np.mean(model_outcomes_coal(propensity_occ, np.array([1, 1, 1, 1, 1, 1]), x_train.drop(columns=['occupation']).iloc[inst,:].values, x_train.drop(columns=['occupation']).values, seed=0))
weight_all_but_occ_but_relationship = np.mean(model_outcomes_coal(propensity_occ, np.array([1, 1, 1, 0, 1, 1]), x_train.drop(columns=['occupation']).iloc[inst,:].values, x_train.drop(columns=['occupation']).values, seed=0))

print('weight_all_but_occ '+str(weight_all_but_occ))
print('weight_all_but_occ_but_relationship '+str(weight_all_but_occ_but_relationship))


/home/clivio/miniconda3/envs/pwshap/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/clivio/miniconda3/envs/pwshap/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


weight_all_but_occ 0.5828651498231744
weight_all_but_occ_but_relationship 0.6143123722980809


In [16]:
phi_mod_by_relationship=((model_outcome_all-model_outcome_all_but_occup)/weight_all_but_occ)-((model_outcome_all_but_relationship-model_outcome_all_but_relationship_but_occup)/weight_all_but_occ_but_relationship)
print('phi_mod_by_relationship' +str(phi_mod_by_relationship))


phi_mod_by_relationship0.222139554073776
